In [ ]:
!pip install tpot==0.11.7
!pip install scikit-learn==1.2.2  
!pip install torch
!pip install --upgrade joblib
!pip install tabulate

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.inspection import permutation_importance
from tpot import TPOTClassifier

np.float = float

print("\nЗавдання 2.1. Підготовка даних")
data = pd.read_csv("train_and_test2.csv")

data.dropna(inplace=True)

X = data.drop(columns=["2urvived"])
y = data["2urvived"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_test_df = pd.DataFrame(X_test, columns=X.columns)

print("\nЗавдання 2.2. Однокритеріальна оптимізація (ROC AUC)")
tpot_roc = TPOTClassifier(
    generations=5,
    population_size=20,
    scoring='roc_auc',
    verbosity=2,
    random_state=42,
    n_jobs=2
)
tpot_roc.fit(X_train, y_train)

roc_auc = roc_auc_score(y_test, tpot_roc.predict_proba(X_test_df)[:, 1])
precision = precision_score(y_test, tpot_roc.predict(X_test_df), zero_division=1)
recall = recall_score(y_test, tpot_roc.predict(X_test_df))

print(f"\nОптимізована модель за ROC AUC:\nROC AUC: {roc_auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

print("\nЗавдання 2.3. Багатокритеріальна оптимізація (Precision + Recall через balanced_accuracy)")
tpot_multi = TPOTClassifier(
    generations=5,
    population_size=20,
    scoring='balanced_accuracy',
    verbosity=2,
    random_state=42,
    n_jobs=2
)
tpot_multi.fit(X_train, y_train)

precision_multi = precision_score(y_test, tpot_multi.predict(X_test_df), zero_division=1)
recall_multi = recall_score(y_test, tpot_multi.predict(X_test_df))
f1_multi = f1_score(y_test, tpot_multi.predict(X_test_df))

print(f"\nБагатокритеріальна оптимізація:\nPrecision: {precision_multi:.4f}, Recall: {recall_multi:.4f}, F1-score: {f1_multi:.4f}")

print("\nЗавдання 2.4. Аналіз Парето-фронту (precision vs recall)")
pareto_data = pd.DataFrame({
    "Модель": ["ROC AUC", "Multi-objective"],
    "Precision": [precision, precision_multi],
    "Recall": [recall, recall_multi]
})
pareto_data["F1"] = 2 * (pareto_data["Precision"] * pareto_data["Recall"]) / (pareto_data["Precision"] + pareto_data["Recall"])

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pareto_data, x="Precision", y="Recall", hue="Модель", s=150, palette="Set2")
plt.title("Парето-фронт моделей (Precision vs Recall)")
plt.xlabel("Precision")
plt.ylabel("Recall")
plt.grid(True)
plt.legend()
plt.show()

idx_best_prec = pareto_data["Precision"].idxmax()
idx_best_rec = pareto_data["Recall"].idxmax()
idx_best_f1 = pareto_data["F1"].idxmax()

print("\nМоделі з Парето-фронту:")
print("Модель з найкращим Precision:", pareto_data.iloc[idx_best_prec]["Модель"])
print("Модель з найкращим Recall:", pareto_data.iloc[idx_best_rec]["Модель"])
print("Модель з найкращим F1-score:", pareto_data.iloc[idx_best_f1]["Модель"])

print("\nЗавдання 2.5. Аналіз важливості ознак")
for model, name in zip([tpot_roc, tpot_multi], ["ROC AUC", "Multi-objective"]):
    importance = permutation_importance(model.fitted_pipeline_, X_test_df, y_test, n_repeats=10, random_state=42)
    feature_importances = pd.DataFrame({
        "Ознака": X.columns,
        "Важливість": importance.importances_mean
    }).sort_values(by="Важливість", ascending=False)

    print(f"\nВажливість ознак для моделі ({name}):\n", feature_importances)

print("\nЗавдання 2.6. Висновки та рекомендації")
from tabulate import tabulate

results = pd.DataFrame({
    "Метод": ["Однокритеріальна (ROC AUC)", "Багатокритеріальна (Precision & Recall)"],
    "ROC AUC": [f"{roc_auc:.4f}", "-"],
    "Precision": [f"{precision:.4f}", f"{precision_multi:.4f}"],
    "Recall": [f"{recall:.4f}", f"{recall_multi:.4f}"],
    "F1-score": ["-", f"{f1_multi:.4f}"]
})

print("\nПорівняльна таблиця моделей:")
print(tabulate(results, headers='keys', tablefmt='grid', showindex=False))

